In [38]:
import pandas as pd
from astropy.io import fits
import numpy as np
import astropy.units as u
from astropy.coordinates import SkyCoord
from tqdm import  tqdm
def match(df_1: pd.DataFrame, df_2: pd.DataFrame, pixel: int, df1_name: list, resolution=0.262) -> pd.DataFrame:
    """
    match two catalog.
    suggestion: df_1 is the real coordinates locally, df_2 is a DataFrame with something wrong in coordinates
    :param df_1:
    :param df_2:
    :param pixel:
    :param df1_name:
    :param resolution: telescope resolution , arcsec/pixel
    :return: using coord in df_1
    """
    sc1 = SkyCoord(ra=df_1.ra, dec=df_1.dec, unit="degree")
    sc2 = SkyCoord(ra=df_2.ra, dec=df_2.dec, unit="degree")
    idx, d2d, d3d = sc1.match_to_catalog_sky(sc2)
    distance_idx = d2d < (pixel * resolution * u.arcsec)

    sc1_matches = df_1.iloc[distance_idx]
    sc2_matches = df_2.iloc[idx[distance_idx]]

    test = sc1_matches.loc[:].rename(columns={"ra": "%s" % df1_name[0], "dec": "%s" % df1_name[1]})
    test.insert(0, 'ID', range(len(test)))
    sc2_matches.insert(0, 'ID', range(len(sc2_matches)))
    new_df = pd.merge(test, sc2_matches, how="inner", on=["ID"])
    return new_df.drop("ID", axis=1)


In [33]:
hdul = fits.open("/data/public/ChenM/MIFIT/MANGA/nsa_v1_0_1.fits")
ra = np.array(hdul[1].data["RA      "])
dec = np.array(hdul[1].data["DEC     "])
run = np.array(hdul[1].data["RUN     "])
rerun = np.array(hdul[1].data["RERUN   "])
camcol = np.array(hdul[1].data["CAMCOL  "])
field = np.array(hdul[1].data["FIELD   "])
MANGA = pd.DataFrame(np.array((ra, dec, run, rerun, camcol, field)).T, columns=["ra", "dec", "run", "rerun", "camcol", "field"])

In [34]:
MANGA.head()

,ra,dec,run,rerun,camcol,field
0,146.714215072841,-1.0412800233313741,756,301,1,206
1,146.6285851269414,-0.7651620976591762,1239,301,2,175
2,146.63170644325592,-0.9883467023052999,756,301,1,206
3,146.9341288729797,-0.6704053610430766,1239,301,2,177
4,146.9634997327016,-0.5447758299302171,756,301,2,208


In [35]:
with fits.open("/data/public/renhaoye/morphics/dataset/VAGC_MGS-m14_1777--20180116.fits") as hdul:
    ra = np.array(hdul[1].data["RA      "])
    dec = np.array(hdul[1].data["DEC     "])
    MGS = pd.DataFrame(np.array((ra, dec)).T, columns=["ra", "dec"])

In [39]:
new = match(MGS[["ra", "dec"]], MANGA, 2, df1_name=["in_ra", "in_dec"]).drop(columns=["ra", "dec"])
new.rename(columns={"in_ra": "ra", "in_dec": "dec"}, inplace=True)

In [42]:
new.to_csv("/data/public/renhaoye/morphics/dataset/sdss_mgs.csv", index=False)